In [5]:
############################# Importing all req. libreries ############################################
import ipywidgets as ipw
from pymongo import MongoClient
import pandas as pd
from bson.son import SON
from IPython.display import display
from IPython.display import Image
from ipywidgets import HBox


############################# Selecting the Entity IDs  ############################################
ids = pd.read_excel("IDs.xlsx")

client = MongoClient("mongodb://localhost:27017")
database = client["SCM_DB"]
#disconnect()
df = pd.read_excel("IDs.xlsx")
Mob_Name = ipw.Dropdown(
    options=list(df["Mobile Names_Neo4J"].dropna()),
    description = "Select Mobile - ", )
raw_type = ipw.Dropdown(
    options=list(df["Raw type"].dropna()),
    description = "Part type - ", )
HBox([Mob_Name,raw_type])

In [2]:
Mob_Name = Mob_Name.value
raw_type = raw_type.value
if raw_type == "Memory Card" : 
    out = "MC"
    spec = "memory_card"
    imei = "mmc imei"
elif raw_type == "RAM" : 
    out = "RAM"
    spec = "RAM"
    imei = "ram imei"
elif raw_type == "GPU" : 
    out = "GPU"
    spec = "GPU"
    imei = "gpu imei"
elif raw_type == "Chipset" : 
    out = "Chipset"
    spec = "Chipset"
    imei = "chip imei"
elif raw_type == "Primary Camara" : 
    out = "PC"
    spec = "primary_camera"
    imei = "pc imei"
elif raw_type == "Sensor" : 
    out = "Sensor"
    spec = "sensors"
    imei = "sen imei"
elif raw_type == "Battery" : 
    out = "Battery"
    spec = "battery"
    imei = "batt imei"
sub_type = ipw.Dropdown(
    options=list(df[out].dropna()),
    description = "Sub type - ", )
sub_type

Dropdown(description='Sub type - ', options=('8 MP', '16 MP', '5 MP', '13 MP', '12 MP'), value='8 MP')

In [3]:
sub_type = sub_type.value
print(Mob_Name , '-----' ,raw_type , '-----' , sub_type)

Samsung Galaxy A60 ----- Primary Camara ----- 16 MP


In [4]:
collection = database["Mobile_Specifications"]

pipeline = [
    {
        u"$match": {
            u"Mobile_Name": Mob_Name
        }
    }, 
    {
        u"$project": {
            u"Mobile_Name": u"$Mobile_Name",
            u"primary_camera": spec,
            u"mobspec": {
                u"$toString": u"$_id"
            },
            u"_id": 0.0
        }
    }, 
    {
        u"$lookup": {
            u"from": u"Top_Level_Assembly_Colleciton",
            u"localField": u"mobspec",
            u"foreignField": u"Mobile Spec Reference",
            u"as": u"k"
        }
    }, 
    {
        u"$unwind": {
            u"path": u"$k"
        }
    }, 
    {
        u"$project": {
            u"raw_imei": u"$k.Raw Material details."+imei
        }
    }, 
    {
        u"$lookup": {
            u"from": u"Raw_Material_Colleciton",
            u"localField": u"raw_imei",
            u"foreignField": u"Raw Material IMEI number",
            u"as": u"c"
        }
    }, 
    {
        u"$unwind": {
            u"path": u"$c"
        }
    }, 
    {
        u"$project": {
            u"Type": u"$c.Raw Material Description",
            u"Price": {
                u"$arrayElemAt": [
                    u"$c.Raw Material Price",
                    0.0
                ]
            },
            u"Quantity": u"$c.Avalibility",
            u"Sup Name": u"$c.Supplier Location Details.Supplier_Name",
            u"Sup Country": u"$c.Supplier Location Details.Supplier_Country"
        }
    }, 
    {
        u"$match": {
            u"Type": sub_type,
            u"Quantity": {
                u"$gt": 300.0
            }
        }
    }, 
    {
        u"$sort": SON([ (u"Price", 1) ])
    }
]

cursor = collection.aggregate(
    pipeline, 
    allowDiskUse = False
)

li = []
try:
    for doc in cursor:
        if doc['Sup Name'] in li: pass
        else :
            li.append(doc['Sup Name'])
            print(doc)
finally:
    client.close()

{'Type': '16 MP', 'Price': 26.0, 'Quantity': 900, 'Sup Name': 'Muller mobile spare parts GmbH', 'Sup Country': 'Germany'}
{'Type': '16 MP', 'Price': 30.0, 'Quantity': 495, 'Sup Name': 'Magna solutions GmbH', 'Sup Country': 'Germany'}
{'Type': '16 MP', 'Price': 35.0, 'Quantity': 830, 'Sup Name': 'Verzon spare parts US Ltd', 'Sup Country': 'USA'}
{'Type': '16 MP', 'Price': 35.0, 'Quantity': 900, 'Sup Name': 'Tesla innovations Ltd', 'Sup Country': 'USA'}
